# 챗봇 종류

### 1. consol base 챗봇
- 전용 챗봇만 가능
-  request.get으로 사용해서 우리가 웹에 호출


### 2.  다른 api와 연계
- 라인, 카카오톡 (openAPI)등과 연계 가능한 챗봇
- fulfillment기능 사용
- dialogflow에서 우리에게 정보를 주는 것

# Fulfillment = WebHook
- 중간에 가로채는 것
- Dialogflow 에이전트가 의도에 따라 비지니스 로직을 호출
- Dialogflow의 자연어 처리로 처리한 추출한 정보를 사용하여 동적 응답 생성

> user
- 스마트폰, 라인, 카톡, 전화 등

> app, application

> dialoglflow
- 인텐트, 엔티티 분석

> fullfillment
- dialogflow가 우리 서버를 호출-> 계산 후, 다시 구글로 보내면 -> user에세 보잼
- 동적으로 계산하기 위해서


### 이종 시스템간에 연결 가능: web
- 우리가 web서버를 구축하면
- 구글이 client가 되어서 우리 서버에 접속해서 가져감.

> 웹서버 만드는 방법
- 플라스크
- 장고

> json
- 구글 , 웹서버가 서로 주고받는 데이터 양식

# Flask

- 웹 서버 기능 

In [ ]:
from flask import Flask

# 객체 생성
app.Flask(__name__)

# 웹 브라우저에 실행할 때 나오는 화면
# 웹 서버에서 호출받은 후, hello를 보냄. 
@app.route('/')
# 함수이름 아무거나 상관없음.
def home():
    return 'hello~~'

if __name__ == '__main__':
    # 내 서버에 접속하는 것
    app.run(host = '0.0.0.0', # 웹서버에 누구나 들어올 수 있게
           port = 3000,
           debug = True)
    

### debug = True
- 서버.py 코드가 수정되어도 다시 실행할 필요 없음

### 내 브라우저에서 실행할 때는 
- 127.0.0.1/3000으로 접속


### IP 주소: v4(4자리)
> 공인 아이피로 접속해야 하는데
- 사설 아이피: 가까운 LAN과만 연결된 아이피
- LAN(공인 아이피를 할당받음)을 통해서 외부와 접속 가능

> 0.0.0.0: 서버에서 사용
- 모든 아이피에 반응하겠다
- 자기 자신 아이피를 모를 때 자기 아이피를 찾아서 사용해줌
- 어디서든 가능
- 사설 아이피를 알아서 할 수도 있지만, 다른 pc에서는 불가


> 로컬 호스트:127.0.0.1 -> roopback 주소
- 이름을 아이피로 바꾸는 규칙에 명시
- 내부 dns 개념
- 브라우저: 명확하게 접속하는 ip(목적지 주소)를 써야 함.
- roopback 주소: 컴퓨터의 랜카드까지만 갔다옴.

### 데코레이터
- 함수 앞뒤로 파이썬이 자동으로 어떤 코드를 붙여줌
- 특정 함수 앞, 뒤로 특정 기능이 자동으로 들어감
- 함수를 wrapper한다는 의미


### not found
- flask에서 실행하는 오류
- 코드에서 찾을 수 없을 대

## 1. get 방식: url로 보냄

### 연습1. get 파라미터: request.args.get()

In [ ]:
@app.route('/')
def home():
    # name: 브라우저에서 호출하면 그 valud값을 불러옴.
    name = request.args.get('name')
    return 'hello~~' + name

> 브라우저에서 
- http://127.0.0.1:3000/?name=abc
- ? :파라미터 입력할 때
- 브라우저에서 입력한 데이터를 서버에서 받아서 다시 브라우저로 받아짐

In [ ]:
@app.route('/')
def home():
    # name: 브라우저에서 호출하면 그 valud값을 불러옴.
    name = request.args.get('name')
    item = request.args.get('item')
    return 'hello~~' + name + item

> 브라우저에서
- http://127.0.0.1:3000/?name=abc&item=%ED%95%9C%EA%B8%80
- 브라우저에서는 한글을 자동으로 인코딩함
-

###  연습2. 날씨 알려주는 함수 추가(사람이 보기 편하게)

In [ ]:
def getWeather(city):
    url = 'https://search.naver.com/search.naver?query='
    url = url + urllib.parse.quote_plus(city + '날씨')
    print(url)
    bs = BeautifulSoup(urllib.request.urlopen(url).read(), 'html.parser')
    temp = bs.select('span.todaytemp')
    desc = bs.select('p.cast_txt')
    return {'temp':temp[0].text, 'desc':desc[0].text}

@app.route('/weather')
def weather():
    city = request.args.get('city')
    info = getWeather(city)
    return '<font color=red>'+info['temp']+'도  ' + info['desc'] +'>/font>' # 출력은 무조건 문자열


> 브라우저에서
- http://127.0.0.1:3000/weather?city=%EB%B6%80%EC%82%B0
- 실제 서버에서 보낸 문자열만 네트워크로  보냄    
- 소스코드를 보내지 않음 
- 서버에서 모두 실행 => 서버 사이드 
- 브라우저: 실행된 결과값만 보냄

### 연습3. 날씨 알려주는 함수(diologflow를 이용하기 위해 json이용)

> 웹 표준 통신 포멧
- json(문자열)

#### json.dumps() 이용

In [ ]:

@app.route('/weather')
def weather():
    city = request.args.get('city')
    info = getWeather(city)
    return json.dumps(info)


> 브라우저에서 결과
- {"temp": "5", "desc": "\ub9d1\uc74c, \uc5b4\uc81c\ubcf4\ub2e4 4\u02da \ub0ae\uc544\uc694"}



#### jsonify()이용 : 더 많이 사용

In [ ]:
@app.route('/weather')
def weather():
    city = request.args.get('city')
    info = getWeather(city)
    return jsonify(info)


> 브라우저 결과
- {
  "desc": "\ub9d1\uc74c, \uc5b4\uc81c\ubcf4\ub2e4 4\u02da \ub0ae\uc544\uc694", 
  "temp": "5"
}

# 서버 호출 해보기

In [17]:
import urllib
from flask import Flask, request, jsonify
from bs4 import BeautifulSoup

In [18]:
city = '서울'
url = 'http://127.0.0.1:3000/whether?city='
url = url + urllib.parse.quote_plus(city)
bs = BeautifulSoup(urllib.request.urlopen(url).read(), 'html.parser')
# temp = bs.select('span.todaytemp')
bs

HTTPError: HTTP Error 404: NOT FOUND

# dialogflow와 연계

#### 정해진 json파일 규칙으로 보내야 함.
- fullfillmentText 부분을 출력함

#### 정해진 문구로 확인

In [ ]:
@app.route('/dialogflow')
def dialogflow():
    res = {'fulfullmentText': 'Hello'}
    return jsonify(res)

> 브라우저에서 먼저 확인하고
- http://127.0.0.1:3000/dialogflow
-{
  "fulfullmentText": "Hello"
}         

#### 브라우저에서는 get방식으로 실행 가능
#### dialogflow에서는 post 방식으로 실행해야 함

In [ ]:
@app.route('/dialogflow', methods = ['POST'])
def dialogflow():
    res = {'fulfullmentText': 'Hello'}
    return jsonify(res)


> 브라우저에서는 
- 오류가남.
- post 방식으로 바뀌었기 때문에 get 방식으로는 불가

> 해결
- dialogflow, 브라우저에서 모두 가능하려면
- method=['PoST', 'GET']

In [ ]:
@app.route('/dialogflow', methods = ['POST', 'GET'])
def dialogflow():
    res = {'fulfullmentText': 'Hello'}
    return jsonify(res)



#### 우리가 고정ip가 아니라 dialogflow에서 접속할 수 없음.
- https://www.postman.com/ : 클라이언트 역할

> Secure Tunnerls(NGROK)
- 공인 아이피가 아니어도 외부에서 접속가능하게 하는 터널 프로그램
- 방화벽 내부 서버를 외부에서 접속 가능
- 가상적인 도메인네임을 부여함.
- dialogflow와 나(서버) 모두 서버(NGROK)에 접속해서 터널이 생김

> 실행
- ngrok http 3000
- 할당된 도메인: http://e06d21c8.ngrok.io 


> 브라우저에
- http://e06d21c8.ngrok.io/dialogflow   -> 구글에 알려줘야 하는 서버주소
- dialogflow와 ngrod이 연결: port번호 80번 사용
- 우리 서버와 ngrok연결: port번호 : 3000번



### dialog flow에 연결하기(홈페이지)
> fulfillment
- webhook:  abled로 변경
- url: https://e06d21c8.ngrok.io/dialogflow
- https: 암호화된 채널로만 통신해야 함.(NGROK에서 둘다 가능)
- 저장


>  intent
- 모든 인텐트가 서버로 다녀올 필요는 없으므로
- 해당 intent에서 fulfillment옵션에서 enable시킴

> 서버가 죽었으면
- 요청하고 좀 기다렸다가 안오면
- 원래 response를 반환함

> 실행하면
- 클라이언트-우리서버-Ngrok -dialog-머신러닝엔진